In [1]:
import sys

from util.django import restart_django

sys.path.append("../../src")

restart_django(imprimir_versoes=True)

Python 3.12.1 (tags/v3.12.1:2305ca5, Dec  7 2023, 22:03:25) [MSC v.1937 64 bit (AMD64)] on win32
Django 5.0.4


Esse notebook tem como objetivo exemplificar o uso das classes de pessoas e usuários do app common

In [2]:
from common.models import *
from datetime import date
import pandas as pd
from django.core.exceptions import ValidationError

## Pessoa

Pessoa é uma classe abstrata, veremos ela sendo usada em conjunto com `UsuarioGenerico`. Ao criar um usuário é chamado o método `clean` de todas suas superclasses, inclusive de pessoa. Dessa maneira, o `cpf`, no caso da herança de `PessoaFisica`, é validado.

Os campos de pessoa física são apresentados abaixo, dois quais se excluí apenas `password` que é atributo do usuário.

In [3]:
try:
    bindo = UsuarioGenericoPessoaFisica.usuarios.criar_usuario(
        cpf='11484174510',
        password='bingosdingos@1234',
        email='eduardocdesouza@gmail.com',
        telefone='32999781208',
        nome='Eduardo',
        sobrenome='C. de Souza',
        data_nascimento=date(2006, 7, 15),
    )
    print('bindo foi salvo no banco de dados')
except ValidationError as e:
    print(f'bindo não foi salvo pois: {e.args[0]}')

bindo não foi salvo pois: {'codigo': [ValidationError(['CPF inválido'])]}


In [4]:
try:
    bindo = UsuarioGenericoPessoaFisica.usuarios.criar_usuario(
        cpf='59247136170812',
        password='bingosdingos@1234',
        email='eduardocdesouza@gmail.com',
        telefone='32999781208',
        nome='Eduardo',
        sobrenome='C. de Souza',
        data_nascimento=date(2006, 7, 15),
    )
    print('bindo foi salvo no banco de dados')
except ValidationError as e:
    print(f'bindo não foi salvo pois: {e.args[0]}')

bindo não foi salvo pois: {'__all__': [ValidationError(['O codigo de PessoaFisica deve ser um cpf'])]}


In [5]:
try:
    bindo = UsuarioGenericoPessoaFisica.usuarios.criar_usuario(
        cpf='11484174518',
        password='bingosdingos@1234',
        email='eduardocdesouza@gmail.com',
        telefone='32999781208',
        nome='Eduardo',
        sobrenome='C. de Souza',
        data_nascimento=date(2006, 7, 15),
    )
    print('bindo foi salvo no banco de dados')
except ValidationError as e:
    print(f'bindo não foi salvo pois: {e.args[0]}')

bindo foi salvo no banco de dados


Não é possível ter dois usuários com o mesmo `cpf` em um mesmo escopo (o escopo assumido quando não é passado é o escopo padrão), conforme podemos ver abaixo:

In [6]:
try:
    hihihiha = UsuarioGenericoPessoaFisica.usuarios.criar_usuario(
        cpf='11484174518',
        password='nom@1234',
        email='kainocah@gmail.com',
        telefone='32999781205',
        nome='Eduardo 3',
        sobrenome='C. de S2ouza',
        data_nascimento=date(2006, 7, 14),
    )
except Exception as e:
    print(f'{type(e).__name__}: {e}')

IntegrityError: duplicate key value violates unique constraint "common_pessoausuario_codigo_scope_id_0d069aa4_uniq"
DETAIL:  Key (codigo, scope_id)=(11484174518, 1) already exists.



Podemos resgatar usuários com o mesmo `cpf` filtrando pelo `cpf` da `PessoaFisica`, todavia o atributo `cpf` não existe de fato, mas é uma outra forma de acessar o atributo `codigo`, mais genérico de `Pessoa`, que também pode ser usado.

In [7]:
pd.DataFrame(UsuarioGenericoPessoaFisica.usuarios.simple().filter(cpf='11484174518'))

,telefone,email,nome,sobrenome,data_nascimento,scope,cpf
0,32999781208,eduardocdesouza@gmail.com,Eduardo,C. de Souza,2006-07-15,1,11484174518


In [8]:
pd.DataFrame(UsuarioGenericoPessoaFisica.usuarios.filter(cpf='11484174518').values())

,password,last_login,is_superuser,is_staff,is_active,date_joined,codigo,telefone,email,pessoa_usuario_id,usuariogenerico_ptr_id,nome,sobrenome,data_nascimento,cpf,scope
0,pbkdf2_sha256$720000$MyhkVDNdeyOSxVj0Frjreo$+F...,None,False,False,True,2024-07-16 21:55:07.380072+00:00,11484174518,32999781208,eduardocdesouza@gmail.com,3,3,Eduardo,C. de Souza,2006-07-15,11484174518,1


Note que o método `simple` pode ser util, mas não retorna a instância de pessoa física, além de perder outros campos de relacionamentos.

In [9]:
isinstance(UsuarioGenericoPessoaFisica.usuarios.simple().filter(cpf='11484174518')[0], dict)

True

In [10]:
isinstance(UsuarioGenericoPessoaFisica.usuarios.filter(cpf='11484174518')[0], UsuarioGenericoPessoaFisica)

True

Note que `codigo` é `editable=False`, logo não deve ser alterado, embora exista essa possibilidade. Caso altere `codigo` outra linha de `Pessoa` será criada, isso ocasionará erros imprevisíveis e assustadores.

Trabalhar com `PessoaJuridica` é análogo a trabalhar com `PessoaFisica`, exceto que o codinome de `codigo` é `cnpj` e não possuí o campo `data_nascimento`, obviamente a verificação do `codigo` verifica se é um `cnpj` válido. Portanto, não será detalhado.

## UsuarioGenerico

O `UsuarioGenerico` estabelece uma relação com `PessoaUsuario` que define o escopo do usuário e garante que existe somente um usuário com o mesmo `codigo` em cada escopo. De forma simplória, cada usuário é uma pessoa e está em um escopo e é a única pessoa com aquele código naquele escopo, o model `PessoaUsuario` serve como ferramenta para garantir isso, além de ser, no sentido do django, o username do usuário, estando em uma relação de 1 para 1, além de ser chave primária de `UsuarioGenerico`. A interface das classes que herdam de `UsuarioGenerico` não se faz prejudicada, no entanto, em relação às default do django. Apesar de não ser abstrata, a classe `UsuarioGenerico` não deve ter instâncias que não estejam associadas com uma de suas subclasses `UsuarioGenericoPessoaFisica` ou `UsuarioGenericoPessoJuridica`, o único motivo de não ser abstrata é a necessidade de definir uma classe não abstrata como sendo a de usuários nas configurações do django.

As classes `UsuarioGenericoPessoaFisica` e `UsuarioGenericoPessoJuridica` herdam, respectivamente, de `PessoaFisica` e `PessoaJuridica`, além de `UsuarioGenericoPessoa` a qual ambas herdam. Por sua vez, `UsuarioGenericoPessoa` herda de `UsuarioGenericoSimple` que herda de `UsuarioGenerico`. Usuário genérico herda de `AbstractUsuarioGenerico` que herda de `AbstractUserPerScopeWithEmail` e de `Pessoa`. Dessa maneira temos uma estrutura em formato de diamante, todavia não encontramos problemas nessa estrutura pois `PessoaFisica` e `PessoaJuridica` herdam de `Pessoa` e todas as três são abstratas.

### Da criação de novos usuários

A criação de novos usuários será pelas classes `UsuarioGenericoPessoaFisica` ou `UsuarioGenericoPessoaJuridica`, é altamente recomendado o uso do método `criar_usuario` e `criar_superusuario`, alternativamente é possível usar o builder. Isso quando feito em código, quando usando o form, deve herdar dos forms que serão feitos no common.

Tanto no `criar_usuario` como será no form, a senha é hasheada e os dados inseridos validados. Portanto, não utilize métodos alternativos e mandrakes para a criação de usuários, pois eles podem não ser validados ou salvar a senha bruta, ambos seriam péssimos.

In [11]:
escocesa_ltda = UsuarioGenericoPessoaJuridica.usuarios.criar_usuario(
    cnpj='12911507000186',
    password='espelho',
    email='mr@gmail.com',
    telefone='31984735439',
    razao_social='Escocia LTDA',
    nome_fantasia='Escocesa Especular'
)

Ou

In [12]:
builder = UsuarioGenericoPessoaJuridica.usuarios.create_builder()
builder.cnpj = '43694255431512'
builder.password = 'bingos'
builder.email = 'algum@gmail.com'
builder.telefone = '32984735439'
builder.razao_social = 'Bingos Dingos LTDA'
builder.nome_fantasia = 'Bingos o Dingo'
bingos_dingos = builder.build_usuario()

Criando um escopo para um grupo de usuários

In [13]:
from scope_auth.models import Scope

In [14]:
outro_escopo = Scope.scopes.create()

In [15]:
bindo = UsuarioGenericoPessoaFisica.usuarios.criar_usuario(
    cpf='51283834235',
    scope=outro_escopo,
    password='bingosdingos@1234',
    email='bindo@gmail.com',
    telefone='32999781209',
    nome='Bindo',
    sobrenome='Dingo'
)

Alterando o usuário `bindo`, modificando os dados de pessoa e os dados de usuário

In [16]:
bindo.nome = 'Bingo'
bindo.email = 'dem@hotmail.com'
bindo.save()

Somente criando outro usuário em mais um escopo

In [17]:
mais_um_escopo = Scope.scopes.create()

coringa = UsuarioGenericoPessoaFisica.usuarios.criar_usuario(
    cpf='37705858797',
    scope=mais_um_escopo,
    password='senha$#$doiasdf2345',
    email='coringa@gmail.com',
    telefone='32999781209',
    nome='Coringa',
    sobrenome=', O Brincalhão'
)

### Resgatando usuários do banco de dados

Ao resgatar usuários do banco de dados nem sempre será possível recuperá-lo como instância de `UsuarioGenericoPessoaFisica` ou `UsuarioGenericoPessoJuridica`, ou não será possível recuperar um queryset dessa maneira. Portanto, mostraremos como resgatar os dados por essas classes e como realizar a coerção de um queryset da classe base ou de uma instância da classe base para as classes desejadas.

Os campos de `cpf` ou `cnpj` são anotados em todo query set, `scope` também é anotado. Também são fornecidas properties e setters para membros de pessoa. Os métodos de `clean` e `save` chamam tambem os de pessoa e pessoa_usuario, conforme em:

```python
bindo.nome = 'Bingo'
bindo.email = 'dem@hotmail.com'
bindo.save()
```

In [18]:
pd.DataFrame(UsuarioGenericoPessoaJuridica.usuarios.simple())

,telefone,email,razao_social,nome_fantasia,scope,cnpj
0,31984735439,mr@gmail.com,Escocia LTDA,Escocesa Especular,1,12911507000186
1,32984735439,algum@gmail.com,Bingos Dingos LTDA,Bingos o Dingo,1,43694255431512


Filtrando por escopo

In [19]:
pd.DataFrame(UsuarioGenericoPessoa.usuarios.filter(scope=outro_escopo).values())

,password,last_login,is_superuser,is_staff,is_active,date_joined,codigo,telefone,email,pessoa_usuario_id,scope
0,pbkdf2_sha256$720000$CaXwxDGu2rpZIpqWfCSRiK$Yw...,None,False,False,True,2024-07-16 21:55:08.523157+00:00,51283834235,32999781209,dem@hotmail.com,7,2


Pegando uma instância por `cnpj` e `scope`

In [20]:
UsuarioGenericoPessoaJuridica.usuarios.get(cnpj='43694255431512', scope=Scope.scopes.default_scope()).__dict__

{'_state': <django.db.models.base.ModelState at 0x1e3ffc9aa80>,
 'password': 'pbkdf2_sha256$720000$3bAOBCPma5fgKb71KSqHBR$2VBuPqS0WykEXoe2Z9/tDyc60UprfT/Z9K0siw53tBE=',
 'last_login': None,
 'is_superuser': False,
 'is_staff': False,
 'is_active': True,
 'date_joined': datetime.datetime(2024, 7, 16, 21, 55, 8, 105017, tzinfo=datetime.timezone.utc),
 'codigo': '43694255431512',
 'telefone': '32984735439',
 'email': 'algum@gmail.com',
 'pessoa_usuario_id': 6,
 'usuariogenerico_ptr_id': 6,
 'razao_social': 'Bingos Dingos LTDA',
 'nome_fantasia': 'Bingos o Dingo'}

Imagine você tem um queryset de `UsuarioGenerico` mas você quer separar as pessoas físicas das pessoas jurídicas.

In [21]:
qs = UsuarioGenerico.usuarios.all()

In [22]:
pessoas_fisicas = qs.as_usuarios_pessoa_fisica()
pessoas_fisicas[0].nome

'Eduardo'

Para um único objeto

In [23]:
if qs[0].is_pessoa_fisica():
    print(qs[0].as_usuario_pessoa_fisica())
else:
    print(qs[0].as_usuario_pessoa_juridica())

<UsuarioGenericoPessoaFisica: {codigo=11484174518, scope=Scope object (1)}>


In [24]:
pessoas_fisicas = qs.as_usuarios_pessoa_fisica()
pd.DataFrame(pessoas_fisicas.simple())

,telefone,email,nome,sobrenome,data_nascimento,scope,cpf
0,32999781208,eduardocdesouza@gmail.com,Eduardo,C. de Souza,2006-07-15,1,11484174518
1,32999781209,dem@hotmail.com,Bingo,Dingo,None,2,51283834235
2,32999781209,coringa@gmail.com,Coringa,", O Brincalhão",None,3,37705858797


In [25]:
pessoas_juridicas = qs.as_usuarios_pessoa_juridica()
pd.DataFrame(pessoas_juridicas.simple())

,telefone,email,razao_social,nome_fantasia,scope,cnpj
0,31984735439,mr@gmail.com,Escocia LTDA,Escocesa Especular,1,12911507000186
1,32984735439,algum@gmail.com,Bingos Dingos LTDA,Bingos o Dingo,1,43694255431512


Alternativamente

In [26]:
pessoas_fisicas = UsuarioGenericoPessoaFisica.usuarios.from_usuarios_queryset(qs)
pd.DataFrame(pessoas_fisicas.simple())

,telefone,email,nome,sobrenome,data_nascimento,scope,cpf
0,32999781208,eduardocdesouza@gmail.com,Eduardo,C. de Souza,2006-07-15,1,11484174518
1,32999781209,dem@hotmail.com,Bingo,Dingo,None,2,51283834235
2,32999781209,coringa@gmail.com,Coringa,", O Brincalhão",None,3,37705858797


In [27]:
try:
    print(qs[0].as_usuario_pessoa_fisica())
except TypeError as e:
    print(e)

<UsuarioGenericoPessoaFisica: {codigo=11484174518, scope=Scope object (1)}>


In [28]:
try:
    print(qs[0].as_usuario_pessoa_juridica())
except TypeError as e:
    print(e)

O usuário não pode ser convertido pois a pessoa 11484174518 não é uma pessoa jurídica.


Basicamente isso exemplifica como trabalhar com um usuário. Em caso de herança o manager e queryset também deverão ser herdados.

In [29]:
coringa = UsuarioGenericoPessoaFisica.usuarios.criar_usuario(
    cpf='37705858797',
    scope=Scope.scopes.default_scope(),
    password='senha$#$doiasdf2345',
    email='coringa@gmail.com',
    telefone='32999781209',
    nome='Coringa',
    sobrenome=', O Brincalhão'
)